In [2]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('Mi Programa')
sc = SparkContext(conf = conf)

In [3]:
sc

<SparkContext master=local appName=Mi Programa>

In [4]:
lines = sc.textFile('ejemplo.txt')
lines

ejemplo.txt MapPartitionsRDD[1] at textFile at <unknown>:0

In [5]:
lines.count()

242

In [6]:
lines.first()

'{{Infobox software'

In [8]:
lines2 = lines.sample(fraction=0.1, withReplacement=False)

In [9]:
lines2.first()

'| developer              = [[Apache Spark]]'

In [10]:
resultado = lines.filter(lambda line:'Python' in line)

In [11]:
resultado

PythonRDD[5] at RDD at PythonRDD.scala:53

In [12]:
resultado.count()

5

In [14]:
resultado.take(3)

['| programming language   = [[Scala (programming language)|Scala]]<ref>{{cite web |url=https://spark.apache.org/releases/spark-release-2-0-0.html |title=Spark Release 2.0.0 |quote=MLlib in R: SparkR now offers MLlib APIs [..] Python: PySpark now offers many more MLlib algorithms"}}</ref>',
 '| language               = [[Scala (programming_language)|Scala]], [[Java (programming language)|Java]], [[SQL]], [[Python (programming language)|Python]], [[R (programming language)|R]], [[C Sharp (programming language)|C#]], [[F Sharp (programming language)|F#]]',
 'Spark Core is the foundation of the overall project. It provides distributed task dispatching, scheduling, and basic [[I/O interface|I/O]] functionalities, exposed through an application programming interface (for [[Java (programming language)|Java]], [[Python (programming language)|Python]], [[Scala (programming language)|Scala]], [[.NET Core|.NET]]<ref name=":1">{{Citation|title=dotnet/spark|date=2020-09-14|url=https://github.com/d

In [16]:
resultado2 = lines.filter(lambda x:any(i.isdigit() for i in x))

In [17]:
resultado2.take(3)

['| logo                   = [[File:Apache Spark logo.svg|frameless|250px|Spark Logo]]',
 '| released               = [https://github.com/apache/spark/releases/tag/v1.0.0 {{Start date and age|2014|05|26}}]',
 '| latest release version = 3.0.1']

In [18]:
resultado2.count()

103

In [19]:
lines.filter(lambda x:not any(i.isdigit() for i in x)).take(5)

['{{Infobox software',
 '| name                   = Apache Spark',
 '| caption                = ',
 '| author                 = [[Matei Zaharia]]',
 '| developer              = [[Apache Spark]]']

In [20]:
resultado2.persist()

PythonRDD[12] at RDD at PythonRDD.scala:53

In [21]:
import pandas as pd

df = pd.read_csv("Z:/csv/casos_diagnostico_ccaa.csv")

In [22]:
df.head()

,ccaa_iso,fecha,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_ag,num_casos_prueba_elisa,num_casos_prueba_desconocida
0,AN,2020-01-01,0,0,0,0,0,0
1,AR,2020-01-01,0,0,0,0,0,0
2,AS,2020-01-01,0,0,0,0,0,0
3,CB,2020-01-01,0,0,0,0,0,0
4,CE,2020-01-01,1,0,0,1,0,0


In [24]:
from pyspark.sql.types import StringType
from pyspark import SQLContext
sqlContext = SQLContext(sc)

dfspark = sqlContext.read.format('csv').option('header','true').option('inferSchema','true').load('Z:/csv/casos_diagnostico_ccaa.csv')

In [25]:
dfspark.show(2)

+--------+----------+---------+--------------------+------------------------+-------------------+----------------------+----------------------------+
|ccaa_iso|     fecha|num_casos|num_casos_prueba_pcr|num_casos_prueba_test_ac|num_casos_prueba_ag|num_casos_prueba_elisa|num_casos_prueba_desconocida|
+--------+----------+---------+--------------------+------------------------+-------------------+----------------------+----------------------------+
|      AN|2020-01-01|        0|                   0|                       0|                  0|                     0|                           0|
|      AR|2020-01-01|        0|                   0|                       0|                  0|                     0|                           0|
+--------+----------+---------+--------------------+------------------------+-------------------+----------------------+----------------------------+
only showing top 2 rows



In [26]:
dfspark.head(2)

[Row(ccaa_iso='AN', fecha='2020-01-01', num_casos=0, num_casos_prueba_pcr=0, num_casos_prueba_test_ac=0, num_casos_prueba_ag=0, num_casos_prueba_elisa=0, num_casos_prueba_desconocida=0),
 Row(ccaa_iso='AR', fecha='2020-01-01', num_casos=0, num_casos_prueba_pcr=0, num_casos_prueba_test_ac=0, num_casos_prueba_ag=0, num_casos_prueba_elisa=0, num_casos_prueba_desconocida=0)]

In [27]:
dfspark.count()

7296

In [29]:
dfspark = dfspark.sample(fraction=0.001, withReplacement=False)

In [30]:
dfspark.count()

11

In [31]:
df2 = dfspark.na.drop(subset=['num_casos','num_casos_prueba_pcr'])

In [32]:
df3 = df2.filter('num_casos_prueba_pcr is not NULL')

In [34]:
df3.count()

11

In [35]:
df3.printSchema()

root
 |-- ccaa_iso: string (nullable = true)
 |-- fecha: string (nullable = true)
 |-- num_casos: integer (nullable = true)
 |-- num_casos_prueba_pcr: integer (nullable = true)
 |-- num_casos_prueba_test_ac: integer (nullable = true)
 |-- num_casos_prueba_ag: integer (nullable = true)
 |-- num_casos_prueba_elisa: integer (nullable = true)
 |-- num_casos_prueba_desconocida: integer (nullable = true)



In [36]:
import numpy as np
media = np.mean(df3.select('num_casos').collect())

In [37]:
media

285.90909090909093

In [38]:
df3.rdd.getNumPartitions()

1